## Извлечение ключевых слов

Или keyword extraction - это упрощенный вариант задачи саммаризации (иногда это называется реферирование, но мне не очень нравится), т.е. извлечения главной информации из текста. Настоящая саммаризация подразумевает, что главная информация описывается нормальными полными предложениями, но сделать это очень сложно. Извлечь только основные слова проще и задачу решает тоже достаточно хорошо (по нескольким слова сразу приблизительно понятно о чем текст и читать ключевые слова сильно быстрее, чем даже самое хороше саммари). Еще одно преимущество ключевых слов - это то, что их удобно использовать в стандартном поиске (который работает только со словами и не анализирует последовательности). 

Одному тексту разные люди могут приписать разные ключевые слова, поэтому оценивать извлечение ключевых слов сложнее, чем обычную классификации. Нужно либо для каждого текста иметь несколько набор ключевых слов для каждого текста, либо брать тексты с ключевыми словами из разных источников. В любом случае с метрикам нужно быть аккуратными и смотреть больше на соотношение, чем на абсолютные значения. Ну и проверять полученные алгоритмы в реальной задаче или хотя бы на глаз. 

In [0]:
!pip install pymorphy2[fast]

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [0]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [0]:
!wget https://github.com/mannefedov/ru_kw_eval_datasets/archive/master.zip && unzip master.zip

In [0]:
!cd ru_kw_eval_datasets-master && mv data ../data

In [0]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [0]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [0]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [12]:
data.shape

(17266, 6)

In [13]:
data.head(3)

,keywords,title,summary,url,content,abstract
0,"[армия, беспилотники, ввс, военная доктрина, военная техника, новые технологии, оружие, россия, самолет, армия]",Беспилотное будущее: в России разрабатываются конвертоплан и тяжёлый БПЛА,"В России для нужд военных разрабатываются конвертопланы и тяжёлые беспилотные летательные аппараты. Об этом на авиакосмическом салоне МАКС-2017 заявил главком Воздушно-космических сил (ВКС) России Виктор Бондарев. По его словам, в будущем военные беспилотники смогут встраиваться в единую систему управления, что позволит существенно снизить расходы на подготовку операторов. В беседе с RT эксперты отметили, что российские инженеры поставили перед собой весьма непростую задачу.",https://russian.rt.com/russia/article/411398-rossiya-konvertoplat-razrabotka,"Главком ВКС России генерал-полковник Виктор Бондарев заявил, что в стране в интересах военных ведётся разработка конвертопланов и тяжёлых беспилотников. По словам генерала, в будущем беспилотники смогут встраиваться в единую систему управления, что позволит существенно снизить расходы на подготовку операторов этих летательных аппаратов.\n Также по теме \n Удар «медведей»: российские Ту-95МС поразили объекты ИГ в Сирии новейшими крылатыми ракетами \n Стратегические ракетоносцы Ту-95МС («медведь» по классификации НАТО) взлетели с российского аэродрома Энгельс в Саратовской области и,... \n«Беспилотная тематика развивается. Как бы то ни было, обучить лётчика дороже, чем поставить на самолет хороший автопилот. Если уже сейчас оператор может управлять одним-двумя беспилотниками, то со временем, с развитием наземной составляющей, он сможет управлять пятью, а то и десятью аппаратами, поэтому это будет ещё дешевле», — заявил Бондарев.\nКроме того, он отметил, что беспилотная авиация в буду...",NaN
1,"[аргентина, барселона, в россии, в мире, испания, лионель месси, россия, сборная россии по футболу, спорт, спортсмен, тренер, фк зенит, футбол, чемпионат россии, чемпионат мира по футболу 2018 в россии, эксклюзив rt, лига чемпионов уефа, футбол, чм по футболу 2018]","«Россия достойно выступит на чемпионате мира»: футболист «Зенита» Краневиттер о ЧМ-2018, Месси и жизни в Петербурге","Сборная России сможет проявить себя на домашнем чемпионате мира. Об этом в интервью RT заявил полузащитник «Зенита» Матиас Краневиттер. 24-летний аргентинец назвал звезду мирового футбола Лионеля Месси обычным человеком, рассказал о своём прозвище на заре карьеры, а также признался, что получает удовольствие от пребывания в Санкт-Петербурге и «Зените».",https://russian.rt.com/sport/article/463504-kranevitter-intervyu-zenit-messi-rossiya,"«Когда моя семья жила бедно, мне приходилось работать кедди»\n— Расскажите о вашем лучшем футбольном воспоминании.\n— Их очень много. Обычно первое, что приходит на ум, — это как ты ещё ребёнком мечтаешь попасть в профессиональный футбол. Играешь с большим удовольствием, сначала для тебя это как развлечение, а потом оно превращается в ежедневную работу. Мне стоило огромных усилий достичь того уровня, на котором я нахожусь сегодня.\n— Вы ещё увлекаетесь гольфом. Как так получилось? \nЯ живу в городе Йерба-Буэна. Вокруг бесконечные поля для игры в гольф. Когда я был мальчишкой, моя семья жила довольно бедно, мне приходилось работать кедди (помощником, который носит инвентарь для игры в гольф. — \nRT\n). Мне всегда это нравилось, я часто играл с друзьями, с родственниками. Этот спорт мне дал очень много. Сейчас я тоже иногда играю, если появляется свободное время. Гольф очень помог мне в своё время. И я думаю, что он во многом помог моей семье, когда я был подростком.\nПубликация от M...",NaN
2,"[армия, безопасность, владимир путин, высокие технологии, информационная война, минобороны, новые технологии, оборона, президент, россия, сергей шойгу, армия]",«Нам удалось вырваться вперёд»: как Россия превзошла Запад в автоматизации управления обороной,"Три года назад в России начал работу Национальный центр управления обороной РФ (НЦУО). Структура в состав

Каждой статье приписано какое-то количество ключевых слов. **Наша задача - придумать как извлекать точно такой же список автоматически.**
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
          
        if union == 0:
          jac = 0
        else:
          jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [15]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


## Домашнее задание

В семинаре использовался только небольшой кусочек данных. На всех данных пересчитайте baseline (tfidf). 

**Ваша задача - предложить 3 способа побить бейзлайн на всех данных.**

Нет никаких ограничений кроме:

1) нельзя изменять метрику  
2) решение должно быть воспроизводимым  
3) способы дожны отличаться друг от друга не только гиперпараметрами (например, нельзя три раза поменять гиперпарамтры в TfidfVectorizer и сдать работу)  
4) изменение количества извлекаемых слов не является улучшением (выберите одно значение и используйте только его)  

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://forms.gle/GWzewBEpw8qnkv8t8

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

В поисках идей можно почитать обзоры по теме (посмотрите еще статьи, в которых цитируются эти обзоры): https://www.semanticscholar.org/search?year%5B0%5D=2012&year%5B1%5D=2020&publicationType%5B0%5D=Reviews&q=keyword%20extraction&sort=relevance

**Использовать доступные готовые решения тоже можно**. Так что погуглите перед тем, как приступать к заданию. 

# Baseline

In [0]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
morph = MorphAnalyzer()
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [0]:
data['content_norm'] = data['content'].apply(normalize)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
data['content_norm_str'] = data['content_norm'].apply(' '.join)
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

evaluate(data['keywords'], keywords)

Precision -  0.09
Recall -  0.12
F1 -  0.09
Jaccard -  0.05


# Самое банальное(нормализация и векторизация)

In [0]:
!pip install russian-names

In [0]:
from russian_names import RussianNames
some_names = set()
for s in RussianNames(count=10000, output_type='dict').get_batch():
  some_names.add(s['name'])

In [21]:
len(some_names)

102

In [0]:
def updated_normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if len(word) > 3]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
    words = [word for word in words if not word in some_names]

    return words

In [0]:
data['content_updated_norm'] = data['content'].apply(updated_normalize)

In [26]:
ranges = [(1,1), (1,2), (2,2)]
analyzers = ['word', 'char']

data['content_updated_norm_str'] = data['content_updated_norm'].apply(' '.join)
for r in ranges:
  for a in analyzers:
    print(r, a)
    new_tfidf = TfidfVectorizer(ngram_range=r, min_df=5, analyzer=a)
    new_tfidf.fit(data['content_updated_norm_str'])
    new_id2word = {i:word for i,word in enumerate(new_tfidf.get_feature_names())}
    new_texts_vectors = new_tfidf.transform(data['content_updated_norm_str'])
    new_keywords = []

    for row in range(new_texts_vectors.shape[0]):
        new_row_data = new_texts_vectors.getrow(row)
        new_top_inds = new_row_data.toarray().argsort()[0,:-11:-1]
        new_keywords.append([new_id2word[w] for w in new_top_inds])

    evaluate(data['keywords'], new_keywords)

(1, 1) word
Precision -  0.09
Recall -  0.12
F1 -  0.09
Jaccard -  0.05
(1, 1) char
Precision -  0.08
Recall -  0.11
F1 -  0.09
Jaccard -  0.05
(1, 2) char
Precision -  0.0
Recall -  0.0
F1 -  0.0
Jaccard -  0.0
(2, 2) word
Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0
(2, 2) char
Precision -  0.0
Recall -  0.0
F1 -  0.0
Jaccard -  0.0


In [27]:
ranges = [(1,1), (1,2), (2,2)]
analyzers = ['word', 'char']

for r in ranges:
  for a in analyzers:
    print(r, a)
    new_tfidf = TfidfVectorizer(ngram_range=r, min_df=5, analyzer=a)
    new_tfidf.fit(data['content_norm_str'])
    new_id2word = {i:word for i,word in enumerate(new_tfidf.get_feature_names())}
    new_texts_vectors = new_tfidf.transform(data['content_norm_str'])
    new_keywords = []

    for row in range(new_texts_vectors.shape[0]):
        new_row_data = new_texts_vectors.getrow(row)
        new_top_inds = new_row_data.toarray().argsort()[0,:-11:-1]
        new_keywords.append([new_id2word[w] for w in new_top_inds])

    evaluate(data['keywords'], new_keywords)

(1, 1) word
Precision -  0.1
Recall -  0.13
F1 -  0.1
Jaccard -  0.06
(1, 1) char
Precision -  0.0
Recall -  0.0
F1 -  0.0
Jaccard -  0.0
(1, 2) word
Precision -  0.09
Recall -  0.12
F1 -  0.09
Jaccard -  0.05
(1, 2) char
Precision -  0.0
Recall -  0.0
F1 -  0.0
Jaccard -  0.0
(2, 2) word
Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0
(2, 2) char
Precision -  0.0
Recall -  0.0
F1 -  0.0
Jaccard -  0.0


Дополнительная нормализация оказалась лишней.
Лучший и победивший бейзалйн набор параметров векторайзера:  
ngram_range=(1, 1), analyzer=word
 

Precision -  0.1  
Recall -  0.13  
F1 -  0.1  
Jaccard -  0.06  

# Опять пробуем графы?

In [0]:
from itertools import combinations
import networkx as nx
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    # менять тут 
    node2measure = dict(nx.degree_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [0]:
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

In [30]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.1
Recall -  0.13
F1 -  0.1
Jaccard -  0.06


Ура?..  
Попробуем другие метрики  
Upd: почему-то ОЧЕНЬ долго считаются, стало грустно и я перестал ждать

In [0]:
def certain_centrality_measure(measure, text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    # менять тут 
    node2measure = dict(measure(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [0]:
measures = [
            nx.betweenness_centrality,
            nx.current_flow_closeness_centrality
]
for m in measures:
  keyword_nx_new = data['content_norm'].apply(lambda x: certain_centrality_measure(m, x, 10, 10))
  evaluate(data['keywords'], keyword_nx_new)
  print()

# Что-то еще?

In [0]:
data['title_norm'] = data['title'].apply(normalize)
data['title_norm_str'] = data['title_norm'].apply(' '.join)

In [0]:
def union_and_intersection(a,b):
  if len(a) == 0:
    return b, b
  if len(b) == 0:
    return a, a
  return list(set(a)|set(b)), list(set(a)&set(b))

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords_content = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords_content.append([id2word[w] for w in top_inds])

tfidf.fit(data['title_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['title_norm_str'])
keywords_title = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords_title.append([id2word[w] for w in top_inds])

In [41]:
unions, intersections = [], []
for i in range(len(keywords_content)):
  un,inter = union_and_intersection(keywords_content[i],keywords_title[i])
  unions.append(un)
  intersections.append(inter)

evaluate(data['keywords'], unions)
evaluate(data['keywords'], intersections)

Precision -  0.06
Recall -  0.15
F1 -  0.08
Jaccard -  0.04
Precision -  0.18
Recall -  0.04
F1 -  0.07
Jaccard -  0.04


In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,1), min_df=5, analyzer='word')

In [0]:
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords_content = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords_content.append([id2word[w] for w in top_inds])

tfidf.fit(data['title_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['title_norm_str'])
keywords_title = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords_title.append([id2word[w] for w in top_inds])

In [51]:
unions, intersections = [], []
for i in range(len(keywords_content)):
  un,inter = union_and_intersection(keywords_content[i],keywords_title[i])
  unions.append(un)
  intersections.append(inter)

evaluate(data['keywords'], unions)
print()
evaluate(data['keywords'], intersections)

Precision -  0.06
Recall -  0.15
F1 -  0.08
Jaccard -  0.05

Precision -  0.18
Recall -  0.05
F1 -  0.07
Jaccard -  0.04


Бейзлайн был такой:
- Precision -  0.09
- Recall -  0.12
- F1 -  0.09
- Jaccard -  0.05

Новые результаты в половине из метрик дают результат хуже, однако:
- результат для объединений:
 - точность упала на 33%
 - полнота выросла на 25%
 - F-мера упала на 11%
 - мера Жаккара не изменилась
- результат для пересечений:
 - точность выросла на 100%
 - полнота упала на 58%
 - F-мера упала на 12%
 - мера Жаккара упала на 20%

Как мне кажется, можно утверждать, что бейзлайн был побежден пересечениями т.к. для ключевых слов самая важная метрика - точность; падение же остальных метрик не так существенно(полнота не так важна, лучше не поставить тег, чем поставить неверный)

In [0]:
def normalize_keywords(words):
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words
    
data['keywords_norm'] = data['keywords'].apply(normalize_keywords)

In [52]:
evaluate(data['keywords_norm'], unions)
print()
evaluate(data['keywords_norm'], intersections)

Precision -  0.08
Recall -  0.24
F1 -  0.11
Jaccard -  0.06

Precision -  0.23
Recall -  0.07
F1 -  0.1
Jaccard -  0.06


Бейзлайн был такой:
- Precision -  0.09
- Recall -  0.12
- F1 -  0.09
- Jaccard -  0.05

А вот нормализовав ключевые слова мы получаем результаты куда лучше:
- результат для объединений:
 - точность упала на 11%
 - полнота выросла на 100%
 - F-мера выросла на 22%
 - мера Жаккара выросла на 20%
- результат для пересечений:
 - точность выросла на 156%
 - полнота упала на 41%
 - F-мера упала на 88%
 - мера Жаккара выросла на 20%

 Но честно ли это? Можем ли мы так делать?..